# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [90]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    (str, int, int, float, int, int, float, str, str, str) -> float / None

    Calculate PRIEST Clinical Severity Score and return the risk percentage.

    >> priest('male', 55, 18, 0.95, 85, 115, 36.9, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
    0.95

    >> priest('female', 70, 23, 0.91, 100, 140, 38.2, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59

    >> priest('unknown', 45, 20, 0.98, 75, 125, 37.3, 'Alert', 'Air', 'Unrestricted normal activity')
    None

    """

    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    sex_score = age_score = rr_score = oxy_score = heart_rate_score = systolic_bp_score = temperature_score = alertness_score = inspired_oxygen_score = performance_status_score = 0

    if sex == 'male':
      sex_score = 1
    elif sex == 'female':
      sex_score = 0
    else :
      return None

    if age >= 16 and age<= 49:
      age_score = 0
    elif age >= 50 and age<= 65:
      age_score = 2
    elif age >=66 and age <= 80:
      age_score = 3
    elif age > 80:
      age_score = 4
    else :
      return None

    if respiratory_rate < 9 or respiratory_rate > 24:
      rr_score = 3
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
      rr_score = 1
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
      rr_score = 2
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
      rr_score = 0
    else:
      return None

    if oxygen_saturation >= 0.95 :
      oxy_score = 0
    elif oxygen_saturation >= 0.94 :
      oxy_score = 1
    elif oxygen_saturation >= 0.92 :
      oxy_score = 2
    elif oxygen_saturation < 0.92:
      oxy_score = 3
    else :
      return None

    if heart_rate >=51 and heart_rate <= 90 :
      heart_rate_score = 0
    elif (heart_rate >=41 and heart_rate <= 50) or (heart_rate >=91 and heart_rate <= 110) :
      heart_rate_score = 1
    elif heart_rate < 41 or heart_rate > 130 :
      heart_rate_score = 3
    elif heart_rate >= 111 and heart_rate <= 130 :
      heart_rate_score = 2
    else :
      return None

    if systolic_bp >= 111 and systolic_bp <= 219 :
      systolic_bp_score = 0
    elif systolic_bp >= 101 and systolic_bp <= 110 :
      systolic_bp_score = 1
    elif systolic_bp >= 91 and systolic_bp <= 100 :
      systolic_bp_score = 2
    elif systolic_bp < 91 and systolic_bp > 219 :
      systolic_bp_score = 3
    else :
      return None

    if temperature >= 36.1 and temperature <= 38 :
      temperature_score = 0
    elif (temperature >= 35.1 and temperature <= 36) or  (temperature >= 38.1 and temperature <= 39) :
      temperature_score = 1
    elif temperature > 39 :
      temperature_score = 2
    elif temperature <35.1 :
      temperature_score = 3
    else :
      return None

    if alertness == 'alert' :
      alertness_score = 0
    elif alertness == 'confused or not alert' :
      alertness_score = 3
    else :
      return None

    if inspired_oxygen == 'air' :
      inspired_oxygen_score = 0
    elif inspired_oxygen == 'supplemental oxygen' :
      inspired_oxygen_score = 2
    else :
      return None

    if performance_status == 'unrestricted normal activity':
      performance_status_score = 0
    elif performance_status == 'limited strenuous activity, can do light activity':
      performance_status_score = 1
    elif performance_status == 'limited activity, can self-care':
      performance_status_score = 2
    elif performance_status == 'limited self-care':
      performance_status_score = 3
    elif performance_status == 'bed/chair bound, no self-care':
      performance_status_score = 4
    else :
      return None

    priest_score = sex_score + age_score + rr_score + oxy_score + heart_rate_score + systolic_bp_score + temperature_score + alertness_score + inspired_oxygen_score + performance_status_score

    if 17 <= priest_score <= 25:
        probability = 0.59
    elif priest_score >= 26:
        probability = 0.99
    else:
        probability = None
    return probability

# Test cases
print(priest('male', 55, 18, 0.95, 85, 115, 36.9, 'confused or not alert', 'supplemental oxygen', 'limited self-care'))
print(priest('female', 70, 23, 0.91, 100, 140, 38.2, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care'))
print(priest('unknown', 45, 20, 0.98, 75, 125, 37.3, 'Alert', 'Air', 'Unrestricted normal activity'))


None
0.59
None


In [91]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [92]:
import requests

def find_hospital(age, sex, risk):
    """
    (int, str, float) -> str

    Find a hospital based on age, sex, and risk percentage using a REST web service.

    >> find_hospital(40, 'male', 0.1)
    Southwest Hospital and Medical Center

    >> find_hospital(35, 'female', 0.05)
    Select Specialty Hospital - Northeast Atlanta

    >> find_hospital(50, 'male', 0.8)
    Emory Dunwoody Medical Center

    """
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            hospital_name = data.get('hospital')
            return hospital_name
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

# Test cases
print(find_hospital(40, 'male', 0.1))  # Example test case
print(find_hospital(35, 'female', 0.05))  # Test case with a different age and risk percentage
print(find_hospital(50, 'male', 0.8))  # Test case with a different age and high risk percentage

Southwest Hospital and Medical Center
Select Specialty Hospital - Northeast Atlanta
Emory Dunwoody Medical Center


In [93]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [94]:
import requests

def get_address(hospital_name):
    """
    (str) -> str

    Get the address of a hospital based on its name from a JSON file.

    >> get_address("COOK MEDICAL CENTER"
    706 NORTH PARRISH AVENUE

    >> get_address("INVALID HOSPITAL")
    None

    >> get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS")
    2000 PALMYRA ROAD
    """
    json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        response = requests.get(json_url)
        if response.status_code == 200:
            hospital_data = response.json()
            if hospital_name in hospital_data:
                return hospital_data[hospital_name]["ADDRESS"]
            else:
                return None
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

# Test cases
print(get_address("COOK MEDICAL CENTER"))  # Test case with a valid hospital name
print(get_address("INVALID HOSPITAL"))  # Test case with an invalid hospital name
print(get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS"))  # Another valid test case


706 NORTH PARRISH AVENUE
None
2000 PALMYRA ROAD


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [97]:
import requests

def process_people(file_location):
    """
    Process patient data from a pipe-delimited file, calculate PRIEST scores, determine hospitals, and get addresses.

    Args:
        file_location (str): URL of the pipe-delimited file containing patient data.

    Returns:
        dict: Dictionary with patient numbers as keys and their information, hospital, and address as values.
    """
    response = requests.get(file_location)

    if response.status_code == 200:
        data = response.text.split('\n')
        processed_data = {}
        for idx, row in enumerate(data):
            # Skip the header row
            if idx == 0:
                continue

            columns = row.strip().split('|')

            if len(columns) != 11:
                print(f"Skipping row {idx + 1} due to incorrect data format.")
                continue

            patient_number, sex, age, breath, o2_sat, heart, systolic_bp, temp, alertness, inspired, status = columns

            # Convert numerical columns to appropriate data types
            age = int(age)
            breath = int(breath)
            o2_sat = float(o2_sat)
            heart = int(heart)
            systolic_bp = int(systolic_bp)
            temp = float(temp)

            risk = priest(sex, age, breath, o2_sat, heart, systolic_bp, temp, alertness, inspired, status)

            hospital = find_hospital(age, sex.lower(), risk)
            address = get_address(hospital) if hospital else None

            patient_info = {
                "sex": sex,
                "age": age,
                "breath": breath,
                "o2_sat": o2_sat,
                "heart": heart,
                "systolic_bp": systolic_bp,
                "temp": temp,
                "alertness": alertness,
                "inspired": inspired,
                "status": status,
                "hospital": hospital,
                "address": address
            }
            processed_data[patient_number] = patient_info

        return processed_data
    else:
        return None

file_location = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
processed_data = process_people(file_location)
print(processed_data)

Skipping row 101 due to incorrect data format.
{'E9559': {'sex': 'FEMALE', 'age': 40, 'breath': 24, 'o2_sat': 0.96, 'heart': 105, 'systolic_bp': 115, 'temp': 34.9, 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'unrestricted normal activity', 'hospital': None, 'address': None}, 'E9385': {'sex': 'Female', 'age': 51, 'breath': 19, 'o2_sat': 0.99, 'heart': 90, 'systolic_bp': 91, 'temp': 38.1, 'alertness': 'ALERT', 'inspired': 'supplemental oxygen', 'status': 'unrestricted normal activity', 'hospital': None, 'address': None}, 'E3067': {'sex': 'female', 'age': 40, 'breath': 29, 'o2_sat': 0.96, 'heart': 105, 'systolic_bp': 95, 'temp': 38.1, 'alertness': 'ALERT', 'inspired': 'air', 'status': 'unrestricted normal activity', 'hospital': None, 'address': None}, 'E9422': {'sex': 'FEMALE', 'age': 66, 'breath': 19, 'o2_sat': 0.96, 'heart': 135, 'systolic_bp': 115, 'temp': 38.1, 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'bed/chair bound, no self-care', 'hospital': None, 'address': None}

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [98]:
file_location = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"

expected_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

response = requests.get(expected_results_url)
if response.status_code == 200:
    expected_results = response.json()
else:
    print("Failed to download expected results.")
    expected_results = {}

processed_data = process_people(file_location)

for patient_number, expected_info in expected_results.items():
    if patient_number in processed_data:
        processed_info = processed_data[patient_number]
        if (
            expected_info[0].lower() == processed_info["sex"].lower()
            and expected_info[1] == processed_info["age"]
            and expected_info[2] == processed_info["breath"]
            and expected_info[3] == processed_info["o2_sat"]
            and expected_info[4] == processed_info["heart"]
            and expected_info[5] == processed_info["systolic_bp"]
            and expected_info[6] == processed_info["temp"]
            and expected_info[7].lower() == processed_info["alertness"].lower()
            and expected_info[8].lower() == processed_info["inspired"].lower()
            and expected_info[9].lower() == processed_info["status"].lower()
            and expected_info[11].lower() == processed_info["hospital"].lower()
            and expected_info[12].lower() == processed_info["address"].lower()
        ):
            print(f"Patient {patient_number}: Data matches the expected results.")
        else:
            print(f"Patient {patient_number}: Data does not match the expected results.")
    else:
        print(f"Patient {patient_number} not found in processed data.")

Skipping row 101 due to incorrect data format.


AttributeError: ignored

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---